In [51]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import requests
from io import BytesIO

from torchvision.models import vgg16_bn, VGG16_BN_Weights
import torch
from torch import nn

In [78]:
def load_picture(link):
    image_url = "https://" + link
    response = requests.get(image_url)

    if response.status_code == 200:
        image_data = response.content
        image = Image.open(BytesIO(image_data)).convert("RGB")
        return image
    else:
        raise Exception


def make_rediction(link, model, device):

    LABELS = {
        0: "clean_photo",
        1: "infographics"
    }

    image = load_picture(link)
    transform = VGG16_BN_Weights.DEFAULT.transforms()
    image = transform(image).to(device).unsqueeze(0)

    model.eval()
    logits = model(image)
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities).item()

    return LABELS[predicted_class]

In [53]:
df = pd.read_csv("../datasets/original_df_for_binary.csv")
df.head()

,pic_url,verdict
0,avatars.mds.yandex.net/get-marketpic/7044394/p...,clean_photo
1,avatars.mds.yandex.net/get-marketpic/5620596/p...,clean_photo
2,avatars.mds.yandex.net/get-marketpic/1708436/m...,infographics
3,avatars.mds.yandex.net/get-marketpic/1897710/p...,clean_photo
4,avatars.mds.yandex.net/get-marketpic/8786714/p...,infographics


In [82]:
df.iloc[23][0]

'avatars.mds.yandex.net/get-marketpic/6097271/pica6718806c7874f7032ed7bb6ff17ed32/orig'

In [79]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = vgg16_bn()
model.classifier = nn.Sequential(
    nn.Linear(25088, 100),
    nn.Softmax(),
    nn.Linear(100, 2)
)

model.load_state_dict(torch.load('../models/model_gr_or_ph.pth', map_location=torch.device(device)))
model = model.to(device)

# link = df.iloc[0][0]
# make_rediction(link, model, device)

for i in range(10):
    link = df.iloc[i][0]
    lable = df.iloc[i][1]
    print(make_rediction(link, model, device), lable)

/Users/yunet/anaconda3/envs/mo_1/lib/python3.10/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


clean_photo clean_photo
clean_photo clean_photo
infographics infographics
clean_photo clean_photo
infographics infographics
infographics clean_photo
infographics infographics
clean_photo clean_photo
infographics infographics
clean_photo clean_photo


In [40]:
!pip install Flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [60]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def hello():
    link = request.args.get('link')  # Получаем значение параметра 'name' из URL
    try:
        r = make_rediction(link, model, device)
        return r
    except Exception as e:
        return f"Неверная ссылка, ошибка: \n {e}"


if __name__ == '__main__':
    app.run(port=2345)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:2345
Press CTRL+C to quit
127.0.0.1 - - [14/Aug/2023 15:15:12] "GET /?link=fd HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2023 15:15:35] "GET /?link=avatars.mds.yandex.net/get-marketpic/4474934/pica5eb45a3d473199e81ce08c251c65109/orig HTTP/1.1" 200 -


In [75]:
%pip freeze | grep -E 'requests' > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [76]:
import PIL
PIL.__version__

'9.4.0'

In [77]:
requests.__version__

'2.29.0'